In [15]:
import os
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    OpenAI.api_key = openai_api_key
client = OpenAI()

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib

import time
import datetime as dt

In [4]:
TWITTER_ACCOUNT = os.getenv("TWITTER_ACCOUNT")
TWITTER_PASSWORD = os.getenv("TWITTER_PASSWORD")

In [5]:
driver = webdriver.Chrome()
driver.get("https://x.com/login/")
driver.maximize_window()
WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.TAG_NAME, "input")))
driver.find_element(By.TAG_NAME,"input").send_keys(TWITTER_ACCOUNT)
driver.find_element(By.XPATH,'//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/button[2]').click()
WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.NAME, "password")))
driver.find_element(By.NAME,"password").send_keys(TWITTER_PASSWORD)
driver.find_element(By.XPATH,'//div/span/span[text()="ログイン"]').click()

In [36]:
target_account_name = "hanairoha_ww"
driver.get("https://x.com/{target_account_name}".format(target_account_name=target_account_name))
WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.TAG_NAME, "article")))

<selenium.webdriver.remote.webelement.WebElement (session="d3c7284787fa371b302a3e2c866f4fa2", element="f.2B4FC6A9DF0F62EC29E5D3964F65F111.d.8C3EB7F7DF391DC394F250BB3F34EF88.e.252")>

In [7]:
def get_text_from_tweet(tweet,target_account_name):
    tweet_text = []
    for i,element_tweet in enumerate(tweet.find_element(By.XPATH,'div/div/div[2]/div[2]').find_elements(By.XPATH,"div")):
        if i == 0:#ツイートのアカウントや日時情報
            tweet_id = element_tweet.find_element(By.XPATH,'div/div/div[1]/div/div[2]/div/div[3]/a').get_attribute("href").split("/")[-1]
        tweet_text.append(element_tweet.text)
    return {"tweet_id":tweet_id,"tweet_text":tweet_text,"html":tweet.get_attribute("innerHTML")}



In [1]:
#ツイートオブジェクトから必要な情報を抜き出して構造化する関数
def get_info_from_tweet(tweet,target_account_name):
    for i,element_tweet in enumerate(tweet.find_element(By.XPATH,'div/div/div[2]/div[2]').find_elements(By.XPATH,"div")):
    # print(i)
    # print(element_tweet.get_attribute("innerHTML"))
        if i == 0:#ツイートのアカウントや日時情報
            account_name = element_tweet.find_element(By.XPATH,'div/div/div[1]/div/div[1]/div//a').get_attribute("href").split("/")[-1]
            if account_name != target_account_name: #プロモツイートを排除
                return False
            time_str = dt.datetime.strftime(
                dt.datetime.strptime(
                element_tweet.find_element(By.TAG_NAME,'time').get_attribute("datetime")
                ,'%Y-%m-%dT%H:%M:%S.%fZ')
                    + dt.timedelta(seconds=60*60*9),'%Y%m%d%H%M%S')
            tweet_id = element_tweet.find_element(By.XPATH,'div/div/div[1]/div/div[2]/div/div[3]/a').get_attribute("href").split("/")[-1]
            # print(time_str)
            # print(tweet_id)
        if i == 2:
            tweet_text = element_tweet.text
            # print(tweet_text)
        if i == 3:
            #一ツイートに複数画像があっても構造はほぼ一緒
            photo_url = []
            for j,photo_link in enumerate(element_tweet.find_elements(By.TAG_NAME,'img')):
                photo_url.append(photo_link.get_attribute("src"))
    return {"time_str":time_str,"tweet_id":tweet_id,"tweet_text":tweet_text,"photo_url":photo_url,"html":tweet.get_attribute("innerHTML")}


In [11]:
#アカウント名とTweetIDからその中身のデータを取得する関数
# def get_info_from_tweet_by_tweetid(account_name,tweet_id):
account_name = "hanairoha_ww"
tweet_id="1845006356631388450"
driver.get("https://x.com/{account_name}/status/{tweet_id}".format(account_name=account_name,tweet_id=tweet_id))

In [16]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
formatted_html = soup.prettify()
print(formatted_html)

<html dir="ltr" lang="ja" style="overflow-y: scroll; overscroll-behavior-y: none; font-size: 15px; color-scheme: dark;">
 <head>
  <style id="draftjs-styles_0">
   .draftjs-styles_0 .public-DraftEditorPlaceholder-root {
      color: #71767B;
    }

    .draftjs-styles_0 .public-DraftEditorPlaceholder-hasFocus {
      color: #71767B;
    }
  </style>
  <style id="draftjs-styles">
   .DraftEditor-editorContainer, .DraftEditor-root, .public-DraftEditor-content {
        height: inherit;
        max-height: inherit;
        min-height: inherit;
        text-align: initial;
    }

    .public-DraftEditor-content[contenteditable=true] {
        -moz-user-modify: read-write-plaintext-only;
        -webkit-user-modify: read-write-plaintext-only;
    }

    .format-inline .public-DraftEditor-content[contenteditable=true] {
      -moz-user-modify: read-write;
      -webkit-user-modify: read-write;
    }

    .DraftEditor-root {
        width: 100%;
        position: relative;
        -webkit-use

In [8]:
#TOPからn件のツイートを取得する関数
def get_new_tweets(account_name, n=5):#固定ツイートを除く処理を分岐で入れる？
    #ブラウザを開いてTwitter開いてログインする処理は一旦スキップ
    driver.get("https://x.com/{account_name}".format(account_name=account_name))
    WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.TAG_NAME, "article")))
    tweets={}
    while True:
        for tweet in driver.find_element(By.XPATH,'//section/div/div/div/div').find_elements(By.TAG_NAME,"article"):
            # tweet_info = get_info_from_tweet(tweet,account_name)
            tweet_info = get_text_from_tweet(tweet,account_name)
            if tweet_info:
                if tweet_info["tweet_id"] not in tweets:
                    tweets[tweet_info["tweet_id"]]=tweet_info#.get_attribute("innerHTML")#.find_element(By.XPATH,'div/div/div[2]/div[2]').find_elements(By.XPATH,"div")
        if len(tweets)>=n:
            break
        driver.execute_script('window.scrollBy(0, 1000);')
        time.sleep(1)
    return tweets

In [9]:
tweets = get_new_tweets("hanairoha_ww")

In [10]:
tweets

{'1838533887288541680': {'tweet_id': '1838533887288541680',
  'tweet_text': ['【公式】花いろは\n@hanairoha_ww\n·\n9月24日',
   '【#花いろは 関西ツアー2024「いろはにほへと」 】\n\n関西ツアーチケット詳細＆ゲスト発表\n\n✦ː──────────\n\n11/2(土)\n＜滋賀＞滋賀U★STONE\nOPEN17:00 / START17:30\nhttps://tiget.net/events/347147\n9/26(木)20:00〜順次\n\n11/3(日)\nさらに表示',
   '',
   '52\n82\n4.8万'],
  'html': '<div class="css-175oi2r r-eqz5dr r-16y2uox r-1wbh5a2"><div class="css-175oi2r r-16y2uox r-1wbh5a2 r-1ny4l3l"><div class="css-175oi2r"><div class="css-175oi2r r-18u37iz"><div class="css-175oi2r r-1iusvr4 r-16y2uox r-ttdzmv"><div class="css-175oi2r r-15zivkp r-q3we1"><div class="css-175oi2r r-18u37iz"><div class="css-175oi2r r-18kxxzh r-1wron08 r-onrtq4 r-obd0qt r-1777fci"><svg viewBox="0 0 24 24" aria-hidden="true" class="r-4qtqp9 r-yyyyoo r-dnmrzs r-bnwqim r-lrvibr r-m6rgpd r-1bwzh9t r-10ptun7 r-1janqcz"><g><path d="M7 4.5C7 3.12 8.12 2 9.5 2h5C15.88 2 17 3.12 17 4.5v5.26L20.12 16H13v5l-1 2-1-2v-5H3.88L7 9.76V4.5z"></path></g></svg></div><div class="c

In [68]:
tweets[list(tweets.keys())[1]].find_element(By.XPATH,'div/div/div[2]/div[2]').find_elements(By.XPATH,"div")

AttributeError: 'str' object has no attribute 'find_element'

In [56]:
element_tweet = tweets[list(tweets.keys())[1]]
print(len(elements_tweet))
for i,element_tweet in enumerate(elements_tweet):
    # print(i)
    # print(element_tweet.get_attribute("innerHTML"))
    if i == 0:#ツイートのアカウントや日時情報
        account_name = element_tweet.find_element(By.XPATH,'div/div/div[1]/div/div[1]/div//a').get_attribute("href").split("/")[-1]
        # if account_name != target_account_name: #プロモツイートを排除
        #     return None
        time_str = dt.datetime.strftime(
            dt.datetime.strptime(
            element_tweet.find_element(By.TAG_NAME,'time').get_attribute("datetime")
            ,'%Y-%m-%dT%H:%M:%S.%fZ')
                + dt.timedelta(seconds=60*60*9),'%Y%m%d%H%M%S')
        tweet_id = element_tweet.find_element(By.XPATH,'div/div/div[1]/div/div[2]/div/div[3]/a').get_attribute("href").split("/")[-1]
        print(time_str)
        print(tweet_id)
    if i == 2:
        tweet_text = element_tweet.text
        print(tweet_text)


StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=129.0.6668.59); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF741ADB645+29573]
	(No symbol) [0x00007FF741A50470]
	(No symbol) [0x00007FF74190B6EA]
	(No symbol) [0x00007FF7419122BE]
	(No symbol) [0x00007FF741914637]
	(No symbol) [0x00007FF7419146F0]
	(No symbol) [0x00007FF74195F38F]
	(No symbol) [0x00007FF74195FA6C]
	(No symbol) [0x00007FF74195312C]
	(No symbol) [0x00007FF74198733F]
	(No symbol) [0x00007FF741952FF6]
	(No symbol) [0x00007FF741987510]
	(No symbol) [0x00007FF7419A86BC]
	(No symbol) [0x00007FF7419870A3]
	(No symbol) [0x00007FF7419512DF]
	(No symbol) [0x00007FF741952441]
	GetHandleVerifier [0x00007FF741E0C58D+3375821]
	GetHandleVerifier [0x00007FF741E57987+3684039]
	GetHandleVerifier [0x00007FF741

In [45]:
tweets = driver.find_element(By.XPATH,'//section/div/div/div/div').find_elements(By.TAG_NAME,"article")
tweet = tweets[0]
for i,element_tweet in enumerate(tweet.find_element(By.XPATH,'div/div/div[2]/div[2]').find_elements(By.XPATH,"div")):
    # print(i)
    # print(element_tweet.get_attribute("innerHTML"))
    if i == 0:#ツイートのアカウントや日時情報
        account_name = element_tweet.find_element(By.XPATH,'div/div/div[1]/div/div[1]/div//a').get_attribute("href").split("/")[-1]
        # if account_name != target_account_name: #プロモツイートを排除
        #     return None
        time_str = dt.datetime.strftime(
            dt.datetime.strptime(
            element_tweet.find_element(By.TAG_NAME,'time').get_attribute("datetime")
            ,'%Y-%m-%dT%H:%M:%S.%fZ')
                + dt.timedelta(seconds=60*60*9),'%Y%m%d%H%M%S')
        tweet_id = element_tweet.find_element(By.XPATH,'div/div/div[1]/div/div[2]/div/div[3]/a').get_attribute("href").split("/")[-1]
        print(time_str)
        print(tweet_id)
    if i == 2:
        tweet_text = element_tweet.text
        print(tweet_text)
    # if i == 3:

20241006090731
1842718291766460430


In [43]:
driver.execute_script('window.scrollBy(0, 1000);')
tweets = driver.find_element(By.XPATH,'//section/div/div/div/div').find_elements(By.TAG_NAME,"article")
print(len(tweets))

1


In [ ]:
#何ツイートか取得して構造を確認する（画像の有無、引用RTの有無、リプか否か）
#本文と画像を取得するコードを書く
#スクロールして何ツイートか取得できるようにする

In [18]:
tweet = tweets[0]

for i,element_tweet in enumerate(tweet.find_element(By.XPATH,'div/div/div[2]/div[2]').find_elements(By.XPATH,"div")):
    print(i)
    print(element_tweet.get_attribute("innerHTML"))
    # if i == 0:#ツイートのアカウントや日時情報
    #     account_name = element_tweet.find_element(By.XPATH,'div/div/div[1]/div/div[1]/div//a').get_attribute("href").split("/")[-1]
    #     if account_name != target_account_name: #プロモツイートを排除
    #         return None
    #     time_str = dt.datetime.strftime(
    #         dt.datetime.strptime(
    #         element_tweet.find_element(By.TAG_NAME,'time').get_attribute("datetime")
    #         ,'%Y-%m-%dT%H:%M:%S.%fZ')
    #             + dt.timedelta(seconds=60*60*9),'%Y%m%d%H%M%S')
    #     tweet_id = element_tweet.find_element(By.XPATH,'div/div/div[1]/div/div[2]/div/div[3]/a').get_attribute("href").split("/")[-1]
    #     print(time_str)
    #     print(tweet_id)
    # #i=1は返信先：ということはリプになっていないとiがずれる？でも陽凪ちゃん全部リプライでした
    # # if i == 2:
    # #     print(element_tweet.text)
    # if i == 3:

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=129.0.6668.59)
Stacktrace:
	GetHandleVerifier [0x00007FF6924FB125+29573]
	(No symbol) [0x00007FF69246FF50]
	(No symbol) [0x00007FF69232B6EA]
	(No symbol) [0x00007FF692312BAC]
	(No symbol) [0x00007FF692312A70]
	(No symbol) [0x00007FF69232E0B1]
	(No symbol) [0x00007FF6923C7FE9]
	(No symbol) [0x00007FF6923A70A3]
	(No symbol) [0x00007FF6923712DF]
	(No symbol) [0x00007FF692372441]
	GetHandleVerifier [0x00007FF69282C76D+3377613]
	GetHandleVerifier [0x00007FF692877B67+3685831]
	GetHandleVerifier [0x00007FF69286CF8B+3641835]
	GetHandleVerifier [0x00007FF6925BB2A6+816390]
	(No symbol) [0x00007FF69247B25F]
	(No symbol) [0x00007FF692477084]
	(No symbol) [0x00007FF692477220]
	(No symbol) [0x00007FF69246607F]
	BaseThreadInitThunk [0x00007FF969F4257D+29]
	RtlUserThreadStart [0x00007FF96C06AF28+40]


In [ ]:
t=0
break_flag = False
while not(break_flag):
    t+=1
    print("t=",t)
    for tweet in driver.find_element(By.XPATH,'//section/div/div/div/div').find_elements(By.TAG_NAME,"article"):
        
        photo_name = get_tweet_tag_photo(tweet)
        if last_photo == photo_name:
            break_flag = True
    driver.execute_script('window.scrollBy(0, 1000);')
    time.sleep(1)

In [11]:

driver.find_element(By.XPATH,'//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/button[2]').click()
